# This notebook uses Papermill to batch run models

In [1]:
%load_ext lab_black
import papermill as pm
import os
from multiprocessing import Pool

## Batch configurations

In [2]:
batch_name = "O2P_main"

# Create batch cfgs
batch_cfgs = []
i = 0

for noise in [0., 1., 2., 4., 8.]:
    for h in [25, 50, 100, 250]:
        for c_unit in [25, 50]:
            if c_unit == 25:
                for c_stage in [0, 195, 300]:
                    # Clean up unit = 25
                    i += 1
                    code_name = batch_name + "_model_{:04d}".format(i)
                    batch_cfg = dict(
                        sn=i,
                        in_notebook="basicOSP_master.ipynb",
                        code_name=code_name,
                        model_folder="models/" + code_name + "/",
                        out_notebook="models/" + code_name + "/output.ipynb",
                        params=dict(
                            code_name=code_name,
                            sample_name='hs04',
                            sample_rng_seed=329,
                            tf_rng_seed=123,
                            use_semantic=False,
                            sem_param_gf=0.,
                            sem_param_gi=0.,
                            sem_param_kf=0.,
                            sem_param_ki=0.,
                            sem_param_hf=0.,
                            sem_param_hi=0.,
                            o_input_dim=119,
                            hidden_units=h,
                            pho_units=250,
                            cleanup_units=c_unit,
                            rnn_activation='sigmoid',
                            regularizer_const=5e-6,
                            embed_attractor_cfg=
                            'models/Attractor_{}/model_config.json'.
                            format(c_unit),
                            embed_attractor_h5='ep{0:04d}.h5'.format(c_stage),
                            p_noise=noise,  # i.e. w_pp, w_pc, and w_cp noise
                            tau=0.2,
                            max_unit_time=4.,
                            n_mil_sample=1.,
                            batch_size=128,
                            learning_rate=0.005,
                            save_freq=5,
                            bq_dataset=batch_name
                        )
                    )
                    batch_cfgs.append(batch_cfg)

            if c_unit == 50:
                for c_stage in [0, 70, 125]:
                    i += 1
                    code_name = batch_name + "_model_{:04d}".format(i)
                    batch_cfg = dict(
                        sn=i,
                        in_notebook="basicOSP_master.ipynb",
                        code_name=code_name,
                        model_folder="models/" + code_name + "/",
                        out_notebook="models/" + code_name + "/output.ipynb",
                        params=dict(
                            code_name=code_name,
                            sample_name='hs04',
                            sample_rng_seed=329,
                            tf_rng_seed=123,
                            use_semantic=False,
                            sem_param_gf=0.,
                            sem_param_gi=0.,
                            sem_param_kf=0.,
                            sem_param_ki=0.,
                            sem_param_hf=0.,
                            sem_param_hi=0.,
                            o_input_dim=119,
                            hidden_units=h,
                            pho_units=250,
                            cleanup_units=c_unit,
                            rnn_activation='sigmoid',
                            regularizer_const=5e-6,
                            embed_attractor_cfg=
                            'models/Attractor_{}/model_config.json'.
                            format(c_unit),
                            embed_attractor_h5='ep{0:04d}.h5'.format(c_stage),
                            p_noise=noise,  # i.e. w_pp, w_pc, and w_cp noise
                            tau=0.2,
                            max_unit_time=4.,
                            n_mil_sample=1.,
                            batch_size=128,
                            learning_rate=0.005,
                            save_freq=5,
                            bq_dataset=batch_name
                        )
                    )
                    batch_cfgs.append(batch_cfg)


# Run
def run_batch(cfg):
    try:
        print("Running model {}".format(cfg['sn']))

        if not os.path.exists(cfg['model_folder']):
            os.mkdir(cfg['model_folder'])

        pm.execute_notebook(
            cfg['in_notebook'],
            cfg['out_notebook'],
            parameters=cfg['params'],
        )

    except:
        print("Error occur in {}".format(cfg['code_name']))

## Run batch

In [ ]:
# Run in parallel pool
# with Pool(4) as pool:
#     pool.map(run_batch, batch_cfgs)

# Push results to BQ
from meta import model_cfg, connect_gbq
import pandas as pd

# Make connection to bq
bq = connect_gbq()

for sn in range(1, len(batch_cfgs) + 1):

    model_folder = 'models/{0:s}_model_{1:04d}'.format(batch_name, sn)
    
    # Load model config
    cfg = model_cfg(None)
    cfg.load_cfg_json(model_folder + '/model_config.json')
    cfg.bq_dataset = batch_name
    
    # Load Strain and Grain
    strain_i_hist = pd.read_csv(model_folder + '/result_strain_item.csv')
    grain_i_hist = pd.read_csv(model_folder + '/result_grain_item.csv')
    bq.push_all(cfg, strain_i_hist, grain_i_hist)


Writing data to Bigquery




1it [00:02,  2.72s/it][A


0it [00:00, ?it/s]

1it [00:12, 12.60s/it]


0it [00:00, ?it/s]

1it [00:10, 10.22s/it]


Completed
Writing data to Bigquery




1it [00:02,  2.13s/it][A


0it [00:00, ?it/s]

1it [00:12, 12.52s/it]


0it [00:00, ?it/s]

1it [00:09,  9.53s/it]


Completed
Writing data to Bigquery


...



1it [00:03,  3.30s/it][A


0it [00:00, ?it/s]

1it [00:13, 13.13s/it]


0it [00:00, ?it/s]

1it [00:06,  6.21s/it]


Completed
Writing data to Bigquery




1it [00:02,  2.94s/it][A


0it [00:00, ?it/s]

1it [00:07,  7.49s/it]


0it [00:00, ?it/s]

1it [00:17, 17.26s/it]


Completed
Writing data to Bigquery




1it [00:02,  2.16s/it][A


0it [00:00, ?it/s]

1it [00:18, 18.14s/it]


0it [00:00, ?it/s]

1it [00:01,  1.17s/it]

#### Shutdown compute engine

In [ ]:
!sudo poweroff  

## Compile results

In [ ]:
import os, json
import pandas as pd
from meta import read_bq_cfg
from evaluate import vis
from IPython.display import clear_output

# batch_name = 'O2P_weightdecay'
save_fig = None
cfgs = read_bq_cfg(batch_name)

# Read cfg files from BQ
print('===== Batch level hyperparams (columns that have >1 unique value) =====')
for i, x in enumerate(cfgs.columns):
    if not x == 'code_name':
        if not x == 'uuid':
            if len(cfgs[x].unique()) > 1:
                print(
                    'Column <{}> has these unique values: {}'.format(
                        x, cfgs[x].unique()
                    )
                )

# Parse each run by batch_eval, which aggregate item level data to condition level and merge Grain and Strain into one single file (Using local files instead of BQ, may use BQ for way way more data... >5Gbs I guess)

models_path = [
    'models/' + batch_name + '_model_{0:04d}'.format(i)
    for i in range(1,
                   len(cfgs) + 1)
]
batch_acc = pd.DataFrame()

for i, name in enumerate(models_path):
    this_eval = vis(
        name, 'result_strain_item.csv', 'result_grain_item.csv'
    )  # Eval lesion and grain
    this_eval.parse_cond_df()
    batch_acc = pd.concat([batch_acc, this_eval.cdf], ignore_index=True)

df = pd.merge(batch_acc, cfgs, 'left', 'code_name')
print('Done')

In [ ]:
def cal_attractor_acc(cleanup_units, embed_attractor_h5):
    if cleanup_units == 25:
        if embed_attractor_h5 == 'ep0000.h5':
            acc = 0.
        if embed_attractor_h5 == 'ep0195.h5':
            acc = 0.6
        if embed_attractor_h5 == 'ep0300.h5':
            acc = 0.9

    if cleanup_units == 50:
        if embed_attractor_h5 == 'ep0000.h5':
            acc = 0.
        if embed_attractor_h5 == 'ep0070.h5':
            acc = 0.6
        if embed_attractor_h5 == 'ep0125.h5':
            acc = 0.9

    return acc

In [ ]:
tmp_acc = []

for i in df.index:
    tmp_acc.append(
        cal_attractor_acc(df.cleanup_units[i], df.embed_attractor_h5[i])
    )

df['attactor_acc'] = tmp_acc

## Overview plot

In [ ]:
# Plotting results
import altair as alt

alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

# Selectors for interactions
sel_run = alt.selection(type="multi", on="click", fields=["code_name"])
sel_cond = alt.selection(
    type="multi", on="click", fields=["cond"], bind="legend"
)

# Heatmap for final epoch & timestep (Overview)
plot_timestep = df.timestep.max()
# plot_timestep = 19

# Plot strain

df_epo_time = df[(df.epoch == df.epoch.max()) & (df.timestep == plot_timestep) &
                 (df.exp == 'strain')]

overview = (
    alt.Chart(df_epo_time).mark_rect().encode(
        x="hidden_units:O",
        y="w_pp_noise:O",
        column='attactor_acc',
        row='cleanup_units',
        color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_run)
)

# Accuracy over epoch at last time step for selected model
last_time_point = df[df.timestep == df.timestep.max()]

acc_plot = (
    alt.Chart(last_time_point).mark_line().encode(
        y=alt.Y("acc:Q", scale=alt.Scale(domain=(0, 1))),
        x="epoch",
        color="cond",
        opacity=alt.condition(sel_cond, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_cond).transform_filter(sel_run).properties(
        title="Full model at final time step"
    )
)

plot = overview | acc_plot
plot

### Save plot

In [ ]:
plot.save('batch_eval/O2P_l2reg.html')

## Eval one model

In [ ]:
from meta import model_cfg
from evaluate import vis

code_name = '{0:}_model_{1:04d}'.format(batch_name, 47)

# Load cfg from json
cfg = model_cfg(None)
cfg.load_cfg_json('models/' + code_name + '/model_config.json')

vis = vis(
    cfg.path_model_folder, 'result_strain_item.csv', 'result_grain_item.csv'
)

vis.parse_cond_df()

full = vis.plot_dev_interactive('acc').properties(title='Full input')

full

### Flexible development plot

In [ ]:
vis.plot_dev('acc', exp=None, condition='cond', timestep=2)

### Flexible time plot

In [ ]:
vis.plot_time('acc', exp='strain', condition='cond', epoch=400)

# Archive for other batches (for record)

## semantic_lr_batch
- Purpose: Examine learning rate effect interaction with g and k
- Hyperparams: lr, g, k
- Modeling branch: OSP (Feedforward)
- Total models: 12

In [ ]:
import os, json
import pandas as pd
from meta import read_bq_cfg
from evaluate import vis
from IPython.display import clear_output

# batch_name = 'over_ride_batch_name_here'
save_fig = None
cfgs = read_bq_cfg(batch_name)

# Read cfg files from BQ
print('===== Batch level hyperparams (columns that have >1 unique value) =====')
for i, x in enumerate(cfgs.columns):
    if not x == 'code_name':
        if not x == 'uuid':
            if len(cfgs[x].unique()) > 1:
                print(
                    'Column <{}> has these unique values: {}'.format(
                        x, cfgs[x].unique()
                    )
                )

# Parse each run by batch_eval, which aggregate item level data to condition level and merge Grain and Strain into one single file (Using local files instead of BQ, may use BQ for way way more data... >5Gbs I guess)

models_path = ['models/' + batch_name + '_model_{0:04d}'.format(i) for i in range(1, 13)]
batch_acc = pd.DataFrame()

for i, name in enumerate(models_path):
    # for i, name in enumerate(os.listdir("models/")):
    this_eval = vis(name, 'result_strain_ns_item.csv', 'result_grain_item.csv') # Eval lesion and grain
    this_eval.parse_cond_df()
    batch_acc = pd.concat(
        [batch_acc, this_eval.cdf], ignore_index=True
    )

df = pd.merge(batch_acc, cfgs, 'left', 'code_name')
print('Done')

# Plotting results
import altair as alt

alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

# Selectors for interactions
sel_run = alt.selection(type="multi", on="click", fields=["code_name"])
sel_cond = alt.selection(
    type="multi", on="click", fields=["cond"], bind="legend"
)

# Heatmap for final epoch & timestep (Overview)
final_epo_time = df[(df.epoch == df.epoch.max()) &
                    (df.timestep == df.timestep.max())]

overview = (
    alt.Chart(final_epo_time).mark_rect().encode(
        y="sem_param_kf:O",
        x="sem_param_gf:O",
        column="learning_rate:Q",
        row = 'exp',
        color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_run)
)

# Accuracy over epoch at last time step for selected model
last_time_point = df[df.timestep == df.timestep.max()]

acc_plot = (
    alt.Chart(last_time_point).mark_line().encode(
        y="acc:Q",
        x="epoch",
        color="cond",
        opacity=alt.condition(sel_cond, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_cond).transform_filter(sel_run).properties(
        title="At final time step"
    )
)

plot = overview | acc_plot
plot.save('batch_eval/batch_lr_g_k_12.html')
plot

## Batch 3
- Purpose: Extra plotting for Pretrain attractor
- Hyperparams: hidden, pretrain or not
- Modeling branch: OSP
- Total models: 8

In [ ]:
import os, json
import pandas as pd
from meta import batch_eval, read_bq_cfg
from IPython.display import clear_output

# batch_name = 'over_ride_batch_name_here'
save_fig = None
cfgs = read_bq_cfg(batch_name)

# Read cfg files from BQ
print('===== Batch level hyperparams (columns that have >1 unique value) =====')
for i, x in enumerate(cfgs.columns):
    if not x == 'code_name':
        if not x == 'uuid':
            if len(cfgs[x].unique()) > 1:
                print(
                    'Column <{}> has these unique values: {}'.format(
                        x, cfgs[x].unique()
                    )
                )

# Parse each run by batch_eval, which aggregate item level data to condition level and merge Grain and Strain into one single file (Using local files instead of BQ, may use BQ for way way more data... >5Gbs I guess)

models_path = [batch_name + '_model_{0:04d}'.format(i) for i in range(1, 9)]
batch_acc = pd.DataFrame()

for i, name in enumerate(models_path):
    # for i, name in enumerate(os.listdir("models/")):
    if name.startswith(batch_name):
        this_eval = batch_eval(name)
        batch_acc = pd.concat(
            [batch_acc, this_eval.export_result()], ignore_index=True
        )

df = pd.merge(batch_acc, cfgs, 'left', 'code_name')
print('Done')

# Plotting results
import altair as alt

alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

# Selectors for interactions
sel_run = alt.selection(type="multi", on="click", fields=["code_name"])
sel_cond = alt.selection(
    type="multi", on="click", fields=["cond"], bind="legend"
)

# Heatmap for final epoch & timestep (Overview)
final_epo_time = df[(df.epoch == df.epoch.max()) &
                    (df.timestep == df.timestep.max())]

overview = (
    alt.Chart(final_epo_time).mark_rect().encode(
        y="cond:N",
        x="regularizer_const:O",
        color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_run)
)

# Accuracy over epoch at last time step for selected model
last_time_point = df[df.timestep == df.timestep.max()]

acc_plot = (
    alt.Chart(last_time_point).mark_line().encode(
        y="acc:Q",
        x="epoch",
        color="cond",
        opacity=alt.condition(sel_cond, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_cond).transform_filter(sel_run).properties(
        title="At final time step"
    )
)

plot = overview | acc_plot
if save_fig is not None:
    plot.save(save_fig)
plot

## Batch 2
- Purpose: Replicate OP models result
- Hyperparams: p-noise, hidden, and clean up
- Modeling branch: OSP
- Total models: 80

In [ ]:
import altair as alt

alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

sel_run = alt.selection(type="multi", on="click", fields=["code_name"])
sel_cond = alt.selection(
    type="multi", on="click", fields=["cond"], bind="legend"
)

final_epo_time = df[(df.epoch == df.epoch.max()) &
                    (df.timestep == df.timestep.max())]

overview = (
    alt.Chart(final_epo_time).mark_rect().encode(
        y="hidden_units:O",
        x="cleanup_units:O",
        column="w_pp_noise",
        row="exp",
        color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_run)
)

last_time_point = df[df.timestep == df.timestep.max()]

acc_plot = (
    alt.Chart(last_time_point).mark_line().encode(
        y="acc:Q",
        x="epoch",
        color="cond",
        opacity=alt.condition(sel_cond, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_cond).transform_filter(sel_run).properties(
        title="At final time step"
    )
)

plot = overview | acc_plot
plot

## Batch 1
- Purpose: Initial testing
- Hyperparams: p-noise, hidden, and clean up
- Modeling branch: OP
- Total models: 36

In [ ]:
# This is a eval script snapshot for batch1 eval
# Put in OSP folder to run
# Use meta eval block for development

batch_name = "batch_v1"

from meta import read_bq_cfg
cfgs = read_bq_cfg('batch_test')

# Read cfg files from BQ
print('===== Check columns that has multiple unique values =====')
for i, x in enumerate(cfgs.columns):
    if not x == 'code_name':
        if not x == 'uuid':
            if len(cfgs[x].unique()) > 1:
                print('Column: {} has these unique values: {}'.format(
                    x, cfgs[x].unique()))

import os, json
import pandas as pd
from meta import batch_eval
from IPython.display import clear_output

# Read all local results
batch_acc = pd.DataFrame()

from meta import model_cfg

class batch_eval():
    def __init__(self, model_folder):
        from evaluate import training_history, strain_eval, grain_eval, plot_development
        from data_wrangling import my_data
        import altair as plt

        self.model_folder = 'models/' + model_folder
        self.load_config()
        self.read_eval_from_file()
        self.max_epoch = self.strain_i_hist['epoch'].max()

    def load_config(self):
        self.cfg = model_cfg(None)
        self.cfg.load_cfg_json(self.model_folder + '/model_config.json')

    def training_hist(self):
        self.t_hist = training_history(self.cfg.path_history_pickle)
        return self.t_hist.plot_all()

    def read_eval_from_file(self):
        self.strain_i_hist = pd.read_csv(self.model_folder +
                                         '/result_strain_item.csv')
        self.grain_i_hist = pd.read_csv(self.model_folder +
                                        '/result_grain_item.csv')

    def parse_strain_cond_df(self, cond):
        self.strain_i_hist['code_name'] = self.strain_i_hist.model_version
        self.scdf = self.strain_i_hist[[
            'code_name', 'epoch', 'sample_mil', 'timestep', 'unit_time', cond,
            'input_s', 'acc', 'sse'
        ]]
        self.scdf = self.scdf.groupby(['code_name', 'epoch', 'timestep', cond],
                                      as_index=False).mean()
        self.scdf['cond'] = self.scdf[cond]
        self.scdf['exp'] = 'strain'

    def parse_grain_cond_df(self, cond):
        self.grain_i_hist['code_name'] = self.grain_i_hist.model_version
        self.gcdf = self.grain_i_hist[[
            'code_name', 'epoch', 'sample_mil', 'timestep', 'unit_time', cond,
            'input_s', 'acc_acceptable', 'sse_acceptable'
        ]]
        self.gcdf = self.gcdf.rename(columns={
            'acc_acceptable': 'acc',
            'sse_acceptable': 'sse'
        })
        self.gcdf = self.gcdf.groupby(['code_name', 'epoch', 'timestep', cond],
                                      as_index=False).mean()
        self.gcdf['cond'] = self.gcdf[cond]
        self.gcdf['exp'] = 'grain'

    def parse_cond_df(self,
                      cond_strain='condition_pf',
                      cond_grain='condition'):
        self.parse_strain_cond_df(cond_strain)
        self.parse_grain_cond_df(cond_grain)
        self.cdf = pd.concat([self.scdf, self.gcdf], sort=False)

    def plot_dev(self, y, time_step=None):

        if time_step == None: time_step = self.cfg.n_timesteps - 1

        title = '{} at timestep {} / unit time {}'.format(
            y, plot_time_step + 1, self.cfg.unit_time)
        sel = alt.selection(type='single',
                            on='click',
                            fields=['cond'],
                            empty='all')
        plot = alt.Chart(
            self.cdf[lambda x: x['timestep'] == time_step]).mark_line(
                point=True).encode(
                    y=y,
                    x='epoch:Q',
                    color='cond',
                    opacity=alt.condition(sel, alt.value(1), alt.value(0)),
                    tooltip=[
                        'epoch', 'timestep', 'sample_mil', 'acc', 'sse'
                    ]).add_selection(sel).interactive().properties(title=title)

        return plot

    def plot_time(self, y, epoch=None):
        if epoch == None: epoch = self.max_epoch
        title = '{} at epoch {} '.format(y, epoch)
        sel = alt.selection(type='single',
                            on='click',
                            fields=['cond'],
                            empty='all')

        plot = alt.Chart(self.cdf[lambda x: x['epoch'] == epoch]).mark_line(
            point=True).encode(
                y=y,
                x='unit_time:Q',
                color='cond',
                opacity=alt.condition(sel, alt.value(1), alt.value(0)),
                tooltip=[
                    'epoch', 'timestep', 'sample_mil', 'acc', 'sse'
                ]).add_selection(sel).interactive().properties(title=title)

        return plot

    def plots(self,
              mode,
              ys,
              cond_strain='condition_pf',
              cond_grain='condition'):
        # Mode = dev(d) / time(t)
        self.parse_cond_df(cond_strain, cond_grain)

        plots = alt.hconcat()
        for y in ys:
            if mode == 'd':
                plots |= self.plot_dev(y, cond_strain, cond_grain)
            elif mode == 't':
                plots |= self.plot_time(y, self.max_epoch, cond_strain,
                                        cond_grain)
            else:
                print('Use d for development plot, use t for time plot')

        return plots

    def export_result(self):
        self.parse_cond_df()
        return self.cdf.reset_index(drop=True)


# Parse each run by batch_eval, which aggregate item level data to condition level and merge Grain and Strain into one single file

for i, name in enumerate(
    ['batch_v1_model_{0:04d}'.format(i) for i in range(1, 36 + 1)]):
    # for i, name in enumerate(os.listdir("models/")):
    if name.startswith(batch_name):
        this_eval = batch_eval(name)
        batch_acc = pd.concat([batch_acc, this_eval.export_result()],
                              ignore_index=True)

df = pd.merge(batch_acc, cfgs, 'left', 'code_name')
clear_output()
print('Done')

import altair as alt

alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

sel_run = alt.selection(type="multi", on="click", fields=["code_name"])
sel_cond = alt.selection(type="multi",
                         on="click",
                         fields=["cond"],
                         bind="legend")

final_epo_time = df[(df.epoch == df.epoch.max())
                    & (df.timestep == df.timestep.max())]

overview = (alt.Chart(final_epo_time).mark_rect().encode(
    y="hidden_units:O",
    x="cleanup_units:O",
    column="w_pp_noise",
    row="exp",
    color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
    opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
    tooltip=["code_name", "acc"],
).add_selection(sel_run))

last_time_point = df[df.timestep == df.timestep.max()]

acc_plot = (alt.Chart(last_time_point).mark_line().encode(
    y="acc:Q",
    x="epoch",
    color="cond",
    opacity=alt.condition(sel_cond, alt.value(1), alt.value(0.1)),
    tooltip=["code_name", "acc"],
).add_selection(sel_cond).transform_filter(sel_run).properties(
    title="At final time step"))

plot = overview | acc_plot
plot.save('batch_eval.html')
